In [2]:
import os
import yaml
import soundfile as sf
import numpy as np

base = 'slakh2100_flac_redux'
splits = ['validation', 'test']
for split in splits:
    split_dir = os.path.join(base, split)
    for track in os.listdir(split_dir):
        track_dir = os.path.join(split_dir, track)
        meta_path = os.path.join(track_dir, 'metadata.yaml')
        stems_dir = os.path.join(track_dir, 'stems')
        out_path = os.path.join(track_dir, 'guitar.flac')
        if not os.path.exists(meta_path):
            continue
        with open(meta_path) as f:
            md = yaml.safe_load(f)
        guitar_files = [os.path.join(stems_dir, stem_id + '.flac')
                        for stem_id, stem in md['stems'].items()
                        if stem.get('inst_class') == 'Guitar' and stem.get('audio_rendered')]
        if not guitar_files:
            continue
        audio_sum = None
        for fpath in guitar_files:
            audio, sr = sf.read(fpath, always_2d=True)
            audio_sum = audio if audio_sum is None else audio_sum + audio
        sf.write(out_path, audio_sum, sr)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'slakh2100_flac_redux\\valid'

In [1]:
import os
import soundfile as sf

def check_audio_durations(base_dir, splits, seq_dur):
    """
    Check audio durations across dataset splits
    """
    short_files = []
    
    for split in splits:
        split_path = os.path.join(base_dir, split)
        if not os.path.exists(split_path):
            print(f"⚠️ Split folder {split} not found")
            continue
            
        print(f"🔍 Checking {split} split...")
        
        for track in os.listdir(split_path):
            track_path = os.path.join(split_path, track)
            mix_file = os.path.join(track_path, "mix.flac")
            
            if not os.path.exists(mix_file):
                continue
                
            try:
                info = sf.info(mix_file)
                duration = info.duration
                
                if duration < seq_dur:
                    short_files.append({
                        'split': split,
                        'track': track,
                        'duration': duration
                    })
                    
            except Exception as e:
                print(f"❌ Error processing {mix_file}: {str(e)}")
    
    # Print summary
    print(f"\n📊 Total tracks shorter than {seq_dur}s: {len(short_files)}")
    if short_files:
        print("\n🚨 Problematic tracks:")
        for file in short_files[:5]:  # Show first 5 examples
            print(f" - {file['split']}/{file['track']}: {file['duration']:.2f}s")
    
    return len(short_files) == 0

# Configuration (adjust these)
BASE_DIR = "slakh2100_flac_redux"  # Or "data" if using the new structure
SPLITS = ["train", "valid", "test"]
SEQ_DUR = 6.0  # Your training sequence duration

# Run the check
all_ok = check_audio_durations(BASE_DIR, SPLITS, SEQ_DUR)

if all_ok:
    print("✅ All audio files meet the duration requirement!")
else:
    print("\n💡 Recommendation: Consider either:")
    print("- Using --seq-dur with a smaller value (current: {SEQ_DUR}s)")
    print("- Padding short files with silence")
    print("- Filtering out tracks shorter than {SEQ_DUR}s")


🔍 Checking train split...
⚠️ Split folder valid not found
🔍 Checking test split...

📊 Total tracks shorter than 6.0s: 0
✅ All audio files meet the duration requirement!


In [26]:
import os
import soundfile as sf
import numpy as np
from pathlib import Path

base_dir = Path('data_slakh')
splits = ['train', 'valid', 'test']

converted_files = []

for split in splits:
    split_dir = base_dir / split
    if not split_dir.exists():
        continue
    for track_dir in split_dir.iterdir():
        if not track_dir.is_dir():
            continue
        for fname in ['mix.flac', 'guitar.flac']:
            fpath = track_dir / fname
            if not fpath.exists():
                continue
            audio, sr = sf.read(fpath, always_2d=True)
            if audio.shape[1] == 1:  # Correct check for mono
                stereo_audio = np.repeat(audio, 2, axis=1)
                sf.write(fpath, stereo_audio, sr)
                converted_files.append(str(fpath))

print(f"Converted {len(converted_files)} mono files to stereo.")
if converted_files:
    print("First few converted files:", converted_files[:5])


KeyboardInterrupt: 

In [3]:
import torch
import torchaudio
import numpy as np
import soundfile as sf
from openunmix import utils, predict

# 1. Load audio
input_path = "separated\htdemucs\Avicii-WakeMeUp\other.wav"
sr = 44100  # Open-Unmix default sample rate
audio, orig_sr = torchaudio.load(input_path)

# Resample if needed
if orig_sr != sr:
    resampler = torchaudio.transforms.Resample(orig_sr, sr)
    audio = resampler(audio)

# Convert to stereo if mono
if audio.shape[0] == 1:
    audio = torch.cat([audio, audio], dim=0)

# Convert to PyTorch tensor and move to the correct device
device = "cuda" if torch.cuda.is_available() else "cpu"
audio_tensor = audio.to(device)

# 2. Ensure correct model paths
model_dir = "open-unmix"  # Path to your trained model (contains guitar.pth and separator.json)
model_name = "guitar"     # The target you trained for

# 3. Load separator model
print(f"Loading separator for model: {model_name} on {device}")
separator = utils.load_separator(
    model_str_or_path=model_dir,
    targets=[model_name],
    niter=1,
    residual=True,  # Enable residual to avoid single target issues
    wiener_win_len=300,
    device=device,
    pretrained=True,
    filterbank="torch",
)

separator.to(device).freeze()
print(f"Separator loaded successfully!")

# 4. Run model on the mixture
estimates = predict.separate(
    audio=audio_tensor,
    rate=sr,
    separator=separator,
    targets=[model_name],
)

# 5. Check if output exists
if model_name not in estimates:
    print(f"No '{model_name}' estimate was generated!")
else:
    # Extract separated guitar
    guitar_estimate = np.squeeze(estimates[model_name].cpu().numpy())

    # Save the separated guitar
    output_path = "Avicii-WakeMeUp_guitar_estimate_demucs.wav"
    sf.write(output_path, guitar_estimate.T, sr)

    print(f"Separated guitar saved at: {output_path}")



Loading separator for model: guitar on cuda
Separator loaded successfully!
Separated guitar saved at: Avicii-WakeMeUp_guitar_estimate.wav


In [2]:
import demucs.separate
import shlex
import os

def run_demucs_other(audio_path):
    # Use spaces and quotes to handle long filenames
    args = shlex.split(f'--two-stems other "{audio_path}"')
    demucs.separate.main(args)
    print("Demucs separation complete. Check the 'separated' folder.")

# Example usage:
run_demucs_other("Avicii-WakeMeUp_guitar_estimate.wav")


Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in D:\FinalThesis\separated\htdemucs
Separating track Avicii-WakeMeUp_guitar_estimate.wav


100%|██████████████████████████████████████████████████████████████████████| 274.95/274.95 [00:17<00:00, 16.05seconds/s]


Demucs separation complete. Check the 'separated' folder.
